In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

In [2]:
import numpy as np

# Load Foil Data

In [3]:
from ICARUS.computation.solvers.XFLR5.polars import read_polars_2d
from ICARUS.database import DB
from ICARUS.core.struct import Struct
from ICARUS.database import EXTERNAL_DB

# Get Plane

In [4]:
from vehicles.Planes.box_wing_nondimensional import get_box_wing
airplane = get_box_wing(name = 'ge_box', AR=9, naca = "0012")

# Import Environment and Set State

In [5]:
from ICARUS.environment.definition import EARTH_ISA
from ICARUS.flight_dynamics.state import State

u_freestream = 20
unstick = State(
    name= "unstick",
    airplane= airplane,
    environment= EARTH_ISA,
    u_freestream= u_freestream
)

# Get Solver

In [6]:
from ICARUS.computation.solvers.AVL.avl import AVL
avl = AVL()
print(avl)

3D VLM Solver AVL:
Available Analyses Are: 
------------------- 
0) Aiplane Polar Analysis 
1) Dynamic Analysis 



## AoA Run

In [7]:
analysis: str = avl.get_analyses_names()[0]
print(f"Selecting Analysis: {analysis}")
avl.select_analysis(analysis)

Selecting Analysis: Aiplane Polar Analysis


In [11]:
options: Struct = avl.get_analysis_options(verbose=True)
solver_parameters: Struct = avl.get_solver_parameters(verbose=True)

Aiplane Polar Analysis
Available Options of AVL for Aiplane Polar Analysis: 

| VarName   | Value                         | Description                                         |
|-----------|-------------------------------|-----------------------------------------------------|
| plane     | Complex Datatype (ge_box_0.0) | Vehicle Airplane Object                             |
| state     | Complex Datatype (unstick)    | State Object                                        |
| solver2D  | Xfoil                         | Name of 2D Solver from which to use computed polars |
| angles    | Complex Datatype              | List of angles to run polars                        |

If there are Multiple Values, or complex datatypes, or N/A you should inspect them sepretly by calling the option name

Available Solver Parameters of AVL for AVL: 

| VarName   | Value   | Description   |
|-----------|---------|---------------|

If there are multiple values you should inspect them sepretly by calling t

In [12]:
AoAmin = -6
AoAmax = 8
NoAoA = (AoAmax - AoAmin) + 1
angles = np.linspace(AoAmin, AoAmax, NoAoA, dtype= float)
# airplane.define_dynamic_pressure(u_freestream, EARTH.air_density)

options.plane         = airplane
options.state         = unstick
options.solver2D      = 'Xfoil'
options.angles        = angles

solver_parameters.inviscid = True

hs = np.linspace(0, 10*airplane.span, 100)

for h in hs:
    unstick.environment._set_altitude(h)
    airplane.name = f"ge_box_{h}"
    avl.define_analysis(options, solver_parameters)
    avl.print_analysis_options()
    avl.execute(parallel = False)
    polars = avl.get_results()
    airplane.save()

Available Options of AVL for Aiplane Polar Analysis: 

| VarName   | Value                         | Description                                         |
|-----------|-------------------------------|-----------------------------------------------------|
| plane     | Complex Datatype (ge_box_0.0) | Vehicle Airplane Object                             |
| state     | Complex Datatype (unstick)    | State Object                                        |
| solver2D  | Xfoil                         | Name of 2D Solver from which to use computed polars |
| angles    | Complex Datatype              | List of angles to run polars                        |

If there are Multiple Values, or complex datatypes, or N/A you should inspect them sepretly by calling the option name

Running Solver AVL:
	Analysis Aiplane Polar Analysis...
Analysis Completed
Getting Results
Available Options of AVL for Aiplane Polar Analysis: 

| VarName   | Value                                          | Description    

# Post Process

In [28]:
import matplotlib.pyplot as plt
from matplotlib import colors


cdict = {
    'red': ((0.0, 0.22, 0.0), (0.5, 1.0, 1.0), (1.0, 0.89, 1.0)),
    'green': ((0.0, 0.49, 0.0), (0.5, 1.0, 1.0), (1.0, 0.12, 1.0)),
    'blue': ((0.0, 0.72, 0.0), (0.5, 0.0, 0.0), (1.0, 0.11, 1.0)),
}

colors_ = colors.LinearSegmentedColormap('custom', cdict)


db3d = DB.vehicles_db
planenames: list[str] = db3d.get_planenames()
fig = plt.figure(figsize=(10,10))
axs = fig.subplots(2,1)  # type: ignore


axs[0].set_xlabel("h [m]")
axs[0].set_ylabel("CL")
axs[0].grid(True)

axs[1].set_xlabel("h [m]")
axs[1].set_ylabel("CD")
axs[1].grid(True)

polars = {}
for planename in planenames:
    if not planename.startswith("ge_box"):
        continue
    # Get h from the name
    h = float(planename.split("_")[-1])

    polar = DB.vehicles_db.polars[planename]
    # Get the dataframe where AoA is between -3 and 3
    polar = polar[(polar['AoA'] >= -3) & (polar['AoA'] <= 3)]

    polars[h] = (polar["AVL CL"], polar["AVL CD"], polar['AoA'])
    
# plot the polars in the same plot.
# We want the keys to be the x axis and the aoa values to be different lines
# For the first plot, we want the CL values
# For the second plot, we want the CD values
    
for i,(h, polar) in enumerate(polars.items()):
    if h == 0:
        h = "Infinity"
    else:
        h = f"{h:.4f}"
    axs[0].plot(polar[2], polar[0],color = colors_(i/len(polars.keys())) , label = f"h = {h} m")
    axs[1].plot(polar[2], polar[1],color = colors_(i/len(polars.keys())) , label = f"h = {h} m")

axs[0].legend()